In [1]:
import pandas as pd
import numpy as np

In [14]:
df = pd.read_csv("cleaned_data.csv")
df_aapl = df[df['RIC']=='AAPL.O']
df_aapl.tail()

,Unnamed: 0,BarTime,Date,Time,RIC,Open,High,Low,Close,VWAP,NumberOfTrades,Volume
98307,98307,2021-09-01 19:55:00+00:00,2021-09-01,15:55:00,AAPL.O,152.430,152.46,152.385,152.4400,152.428,2355,487520.0
98308,98308,2021-09-01 19:56:00+00:00,2021-09-01,15:56:00,AAPL.O,152.440,152.47,152.420,152.4400,152.445,2084,356602.0
98309,98309,2021-09-01 19:57:00+00:00,2021-09-01,15:57:00,AAPL.O,152.450,152.47,152.400,152.4600,152.438,1869,339385.0
98310,98310,2021-09-01 19:58:00+00:00,2021-09-01,15:58:00,AAPL.O,152.460,152.50,152.430,152.4801,152.461,2263,512189.0
98311,98311,2021-09-01 19:59:00+00:00,2021-09-01,15:59:00,AAPL.O,152.485,152.56,152.460,152.5400,152.501,4288,868164.0


#### Calculating log returns

In [15]:
df_aapl['returns'] = (np.log(df_aapl['Close'] / df_aapl['Close'].shift(-1)))

<ipython-input-15-087851d04d66>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_aapl['returns'] = (np.log(df_aapl['Close'] / df_aapl['Close'].shift(-1)))


#### Creating dataframe for AAPL stock prices on 2020-09-01

In [22]:
df_aapl_Day1 = df_aapl.head(390)
df_aapl_Day1['returns_sq'] = df_aapl_Day1['returns'].pow(2) # adding a returns^2 column which is required for the calculation of ewma

<ipython-input-22-ae2a1428330c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_aapl_Day1['returns_sq'] = df_aapl_Day1['returns'].pow(2) # adding a returns^2 column which is required for the calculation of ewma


#### Calculating weights for each row using lambda = 0.94  

In [17]:
df_aapl_Day1['weights'] = pd.Series(weights).values

<ipython-input-17-2411e9a4a878>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_aapl_Day1['weights'] = pd.Series(weights).values


In [18]:
lambd = 0.94
weights = []
weight = 1 - lambd
for i in range(390):
    if i == 0:
        weights.append(weight)
    else:
        weight = weight * lambd
        weights.append(weight)

In [19]:
df_aapl_Day1['vol_ewm'] = df_aapl_Day1['weights'] * df_aapl_Day1['returns']

<ipython-input-19-49c8b4949729>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_aapl_Day1['vol_ewm'] = df_aapl_Day1['weights'] * df_aapl_Day1['returns']


#### Calculating final volitility for Day 1:

In [21]:
ewma = np.sum(df_aapl_Day1['vol_ewm'])
print("Volatility for Day 1 is:", ewma)

Volatility for Day 1 is: 0.00015130974259748028
